# Evolution of $M_{\rm D}$ and $\omega$ over Time

This notebook aims to reproduce Fig. 2 from [Gibson et al. (2017)](https://arxiv.org/abs/1706.04802) which examines how the disc mass $M_{\rm D}$ and the angular frequency $\omega$ evolve over time after `odeint` has solved the ODEs and how the evolution is affected by different values of timescale ratio $\epsilon$ and mass ratio $\delta$. By also examining the fastness parameter evolution, we can observe how the variation in $\epsilon$ and $\delta$ affects the "switch-on" of the propeller regime ($\Omega > 1$).

## 1. Module Imports

Here we import standard libraries `numpy` and `matplotlib` and import `odeint` from `scipy`. Rather than redefine `init_conds` again, it has been exported to a package called `magnetar` and can also be imported.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from magnetar.funcs import init_conds

%matplotlib inline

In [ ]:
# Constants
G = 6.674e-8                      # Gravitational constant - cgs units
c = 3.0e10                        # Speed of light - cm/s
Msol = 1.99e33                    # Solar mass - grams
M = 1.4 * Msol                    # Magnetar mass - grams
R = 1.0e6                         # Magnetar radius - cm
alpha = 0.1                       # Sound speed prescription
cs7 = 1.0                         # Sound speed - cm/s
I = (4.0 / 5.0) * M * (R ** 2.0)  # Moment of Inertia
GM = G * M

In [ ]:
# Model to be passed to ODEINT to calculate evolving disc mass and
# angular frequency
def ODEs(y, t, B, MdiscI, RdiscI, epsilon, delta, n=1.0, alpha=0.1, cs7=1.0,
         k=0.9):
    """
Function to pass to ODEINT which will calculate a disc mass and angular
frequency for given time points.

Usage >>> ODEs(y, t, B, MdiscI, RdiscI, epsilon, delta, n)
      y : output of init_conds()
      t : array of time points (t_arr above)
      B : Magnetic Field Strength - 10^15 Gauss (float)
 MdiscI : Initial disc mass - Solar masses (float)
 RdiscI : Disc radius - km (float)
epsilon : timescale ration (float)
  delta : mass ratio (float)
      n : propeller "switch-on" (float)
  alpha : Viscosity prescription (float)
    cs7 : Sound speed - 10^7 cm/s (float)
      k : capping fraction (float)

Returns an array object containing the time derivatives of the disc mass and
angular frequency to be integrated by ODEINT.
    """
    # Initial conditions
    Mdisc, omega = y
    
    # Constants
    Rdisc = RdiscI * 1.0e5                 # Disc radius - cm
    tvisc = Rdisc / (alpha * cs7 * 1.0e7)  # Viscous timescale - s
    mu = 1.0e15 * B * (R ** 3.0)           # Magnetic Dipole Moment
    M0 = delta * MdiscI * Msol             # Global Fallback Mass Budget - g
    tfb = epsilon * tvisc                  # Fallback timescale - s
    
    # Radii - Alfven, Corotation, Light Cylinder
    Rm = ((mu ** (4.0 / 7.0)) * (GM ** (-1.0 / 7.0)) * (((3.0 * Mdisc / tvisc))
          ** (-2.0 / 7.0)))
    Rc = (GM / (omega ** 2.0)) ** (2.0 / 3.0)
    Rlc = c / omega
    # Cap the Alfven radius
    if Rm >= k * Rlc:
        Rm = k * Rlc
    
    w = (Rm / Rc) ** (3.0 / 2.0)  # Fastness parameter
    
    bigT = 0.5 * I * (omega ** 2.0)  # Rotational energy
    modW = (0.6 * M * (c ** 2.0) * ((GM / (R * (c ** 2.0))) / (1.0 - 0.5 * (GM
            / (R * (c ** 2.0))))))   # Binding energy
    rot_param = bigT / modW          # Rotation parameter
    
    # Dipole torque
    Ndip = (-1.0 * (mu ** 2.0) * (omega ** 3.0)) / (6.0 * (c ** 3.0))
    
    # Mass flow rates and efficiencies
    eta2 = 0.5 * (1.0 + np.tanh(n * (w - 1.0)))
    eta1 = 1.0 - eta2
    Mdotprop = eta2 * (Mdisc / tvisc)  # Propelled
    Mdotacc = eta1 * (Mdisc / tvisc)   # Accreted
    Mdotfb = (M0 / tfb) * ((t + tfb) / tfb) ** (-5.0 / 3.0)  # Fallback rate
    Mdotdisc = Mdotfb - Mdotprop - Mdotacc  # Mass flow through the disc
    
    if rot_param > 0.27:
        Nacc = 0.0  # Prevents magnetar break-u[
    else:
        # Accretion torque
        if Rm >= R:
            Nacc = ((GM * Rm) ** 0.5) * (Mdotacc - Mdotprop)
        else:
            Nacc = ((GM * R) ** 0.5) * (Mdotacc - Mdotprop)
    
    omegadot = (Nacc + Ndip) / I  # Angular frequency time derivative
    
    return np.array([Mdotdisc, omegadot])

## 2. Variable Set-up

Here we set up global constants required for the calculation and the values of $\epsilon$ and $\delta$ we'd like to test.

In [ ]:
B = 1.0         # Magnetic Field Strength - 10^15 Gauss
P = 5.0         # Initial spin period - milliseconds
MdiscI = 0.001  # Initial disc mass - Solar masses
RdiscI = 100.0  # Disc radius - km

mu = 1.0e15 * B * (R ** 3.0)           # Magnetic Dipole Moment
Rdisc = RdiscI * 1.0e5                 # Convert disc radius to cm
tvisc = Rdisc / (alpha * cs7 * 1.0e7)  # Viscous timescale - seconds

t = np.logspace(0.0, 6.0, base=10.0, num=10001)  # Time array

y = init_conds(P, MdiscI)  # Initial conditions

eps_vals = [1.0, 1.0, 10.0, 10.0]  # Epsilon values
delt_vals = [1.0, 10.0, 1.0, 10.0]  # Delta values

## 3. Solve the Equations and Plot the Results

Here we loop over the desired $\epsilon$ and $\delta$ values and plot the figure.

In [ ]:
# Set up a figure with 3 subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(6,7))
ax3.axhline(1.0, ls='--', c='k')  # Marks the w > 1 condition

colours = ['r', 'r', 'g', 'g']  # Line colours
lines = ['-', '--', '-', '--']  # Linestyles

# Loop over the epsilon and delta values
for i, vals in enumerate(zip(eps_vals, delt_vals)):
    epsilon, delta = vals

    # Solve the ODEs and split the solution
    # n value defaults to 1 when absent from args
    soln = odeint(ODEs, y, t, args=(B, MdiscI, RdiscI, epsilon, delta))
    Mdisc = soln[:,0]
    omega = soln[:,1]
    
    # Radii
    Rm = ((mu ** (4.0 / 7.0)) * (GM ** (-1.0 / 7.0)) * ((3.0 * Mdisc) / tvisc)
          ** (-2.0 / 7.0))
    Rc = (GM / (omega ** 2.0)) ** (1.0 / 3.0)
    
    # Plotting
    ax1.loglog(t, Mdisc/Msol, ls=lines[i], c=colours[i],
               label='$\epsilon$ = {0}; $\delta$ = {1}'.format(
                      int(epsilon), int(delta)))
    ax2.semilogx(t, omega, ls=lines[i], c=colours[i])
    ax3.semilogx(t, Rm/Rc, ls=lines[i], c=colours[i])

# Plot formatting
ax1.set_xlim(1.0e0, 1.0e6)
ax1.set_ylim(bottom=1.0e-8)
ax2.set_xlim(1.0e0, 1.0e6)
ax3.set_xlim(1.0e0, 1.0e6)
ax3.set_ylim(0.0, 2.0)

ax1.tick_params(axis='both', which='major', labelsize=8)
ax2.tick_params(axis='both', which='major', labelsize=8)
ax3.tick_params(axis='both', which='major', labelsize=8)

ax1.legend(loc='upper right', fontsize=8)
ax3.set_xlabel('Time (s)', fontsize=10)

ax1.set_ylabel('$M_{\\rm D}~\left({\\rm M}_{\odot}\\right)$', fontsize=10)
ax2.set_ylabel('$\omega~\left({\\rm s}^{-1}\\right)$', fontsize=10)
ax3.set_ylabel('$r_{\\rm m}/r_{\\rm c}$', fontsize=10);

<div class="alert alert-block alert-warning">
<b>Warning:</b> This plot is currently not reproducible from the paper.
</div>